# Pedestrian avoidance system for self-driving car (client-side)

This example demonstrates AdaStress's remote stress testing capabilities with an example scenario: Alice has designed a pedestrian avoidance system for her self-driving car and would like to make use of Bob's stress testing infrastructure. For reasons of confidentiality and/or security, Alice does not wish to share or reveal the system software.

AdaStress provides an interface by which Bob may connect securely to a server belonging to Alice and perform stress testing. An information gap ensures that he receives no information about the system or its behavior, other than the limited information that Alice chooses to share.

In this scenario, Alice chooses the option that affords maximum privacy: a blackbox salted seed-action simulation. No information about the state or action space is shared, and all seeds are rehashed with a private token. This ensures that Bob could not independently reproduce any failures he finds, even given an exact copy of Alice's software.

*This notebook contains the client-side implementation, consisting of the client and solver.*

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating environment at `C:\Users\rlipkis\.julia\dev\adastress\examples\pedestrian\Project.toml`


In [2]:
using AdaStress
using AdaStress.MonteCarloTreeSearch
using Random

┌ Info: Precompiling AdaStress [f8632b6a-8763-4da0-bfaf-5f7707adef25]
└ @ Base loading.jl:1317


In [3]:
function mdp_client()
    client = Interface.ASTClient{Interface.UnobservableState, Interface.SeedAction}()
    Interface.connect!(client, Interface.getipaddr(), 2000)
    return client
end

mdp_client (generic function with 1 method)

In [4]:
mcts = MCTS(num_iterations=10_000, α=0.85)

MCTS(10000, 10, 1.0, 0.85, 1.0, nothing)

In [5]:
Random.seed!(0)
sol = mcts(mdp_client)

┌ Info: Connected to AST server in 0.0320276 seconds.
│   client.conn = Sockets.TCPSocket(Base.Libc.WindowsRawSocket(0x0000000000001130) open, 0 bytes waiting)
└ @ AdaStress.Interface C:\Users\rlipkis\.julia\dev\AdaStress\src\interface\remote\client.jl:33
Progress: 100%|█████████████████████████████████████████| Time: 0:02:21


DataStructures.PriorityQueue{Any, Any, Base.Order.ForwardOrdering} with 10 entries:
  UInt32[0xe5e0c783, 0xc7bb0a3d, 0x57a89aa5, 0xe02f0396, 0x3cdcca20,… => 1084.57
  UInt32[0xe5e0c783, 0xc7e4cd8a, 0x4c80391e, 0x1021b9d2, 0xbcc4cbd5,… => 1084.6
  UInt32[0xe5e0c783, 0xe535ba11, 0xdd294da9, 0xd8dbd174, 0xe7ebc2fd,… => 1084.61
  UInt32[0xe5e0c783, 0xb3118e31, 0x1dd32a10, 0x517971ca, 0xcd0a4974,… => 1084.7
  UInt32[0xe5e0c783, 0xe535ba11, 0x4dad7963, 0xabe979b4, 0xfede8c05,… => 1084.84
  UInt32[0xe5e0c783, 0x7f9e1498, 0x673b6fa1, 0x8b7ec923, 0xfb6e0c1c,… => 1084.97
  UInt32[0xe5e0c783, 0xe62d20d1, 0x1bd877d5, 0x2412d807, 0xac738344,… => 1085.08
  UInt32[0xe5e0c783, 0xa515af0a, 0x412e94a8, 0x6def48c7, 0x0bb3d802,… => 1086.73
  UInt32[0xe5e0c783, 0x710fdc45, 0x08a9a46b, 0x90c278e4, 0x2a5dacce,… => 1086.98
  UInt32[0xe5e0c783, 0xc7bb0a3d, 0x96cbaddc, 0xdbebcc3c, 0xa41602e3,… => 1087.03

In [6]:
@show MonteCarloTreeSearch.total_size(mcts.tree);
@show MonteCarloTreeSearch.max_depth(mcts.tree);

MonteCarloTreeSearch.total_size(mcts.tree) = 10000
MonteCarloTreeSearch.max_depth(mcts.tree) = 8


In [7]:
using BSON

In [8]:
BSON.@save "best_paths.bson" sol